In [ ]:
from capella2polarion.connectors import polarion_worker
from capella2polarion.converters import document_renderer
import dotenv
import os
import capellambse

Create a `.env` file with the following values:
- MODEL_PATH
- POLARION_PROJECT
- POLARION_HOST
- POLARION_PAT

In [ ]:
dotenv.load_dotenv()
model = capellambse.MelodyModel(os.environ.get("MODEL_PATH", ""))
worker = polarion_worker.CapellaPolarionWorker(
    polarion_worker.PolarionWorkerParams(
            os.environ.get("POLARION_PROJECT", ""),
            os.environ.get("POLARION_HOST", ""),
            os.environ.get("POLARION_PAT", ""),
            delete_work_items=False,
        )
)
worker.load_polarion_work_item_map()

In [ ]:
renderer = document_renderer.DocumentRenderer(worker.polarion_data_repo, model)

If the document, we want to render already exists in Polarion, we should request it before we re-render it.

In [ ]:
old_doc=worker.client.project_client.documents.get("_default", "TEST-ICD1", fields={"documents":"@all"})

In this example we want to create a document to describe an interface. The template expects the UUID of a component exchange. As we know that the document already exists in Polarion, we pass it to the renderer, to let it reuse existing heading workitems. The workitems which should be updated are returned in addition to the newly generated document.

In [ ]:
new_doc, wis=renderer.render_document(
    "document_templates",
    "test-icd.html.j2",
    document=old_doc,
    interface="3d21ab4b-7bf6-428b-ba4c-a27bca4e86db"
)

In [ ]:
worker.client.project_client.work_items.update(wis)
worker.client.project_client.documents.update(new_doc)

If we want to create a new document, we don't have to pass a document

In [ ]:
new_doc, wis=renderer.render_document(
    "document_templates",
    "test-icd.html.j2",
    "_default",
    "TEST-ICD6",
    interface="3d21ab4b-7bf6-428b-ba4c-a27bca4e86db"
)

In [ ]:
worker.client.project_client.documents.create(new_doc)